#  **Capstone: Battle of the Neighbourhoods**

### <font color='grey'>Segmenting and Clustering Calgary Neighbourhoods Based on Current Real Estate and Location Information</font>

<font color='grey'>_Completed to fulfill requirements for the IBM Data Science Professional Certificate_</font>

## **Introduction**

Finding the perfect home can be a challenge as there are many different factors to consider when purchasing a home. This project aims to help buyers find their ideal neighbourhoods based on real estate data such as type of house and price as well as using neighbourhood location data to evaluate what kind of venues are close by.

The area of focus for this project will be in Calgary, AB although the methodology will be applicable to any city.

Data science will be implemented to segment and cluster neighbourhoods using information obtained from the real estate website realtor.ca and location information retrieved using four square. The K-means clustering technique will be used because the data can be separated into defined categories that can be represented numerically. These clusters will be used to identify different types of buyers and the neighbourhoods best suited for their purposes.

## **Data**

In order to investigate this problem the following data will be needed:
1. Real estate data such as price, house type, size, etc
2. Calgary Neighbourhood list and coordinates
3. Neighbourhood venue information    

This data will be retrieved respectively using the following:
1. Real Estate data will be scraped from the realtor.ca website using the real estate API 
2. Calgary Neighbourhoods will be manually saved from wikipedia into a CSV and then loaded in as a dataframe
3. Neighbourhood venue information will be retrieved using Four Square API



### Data Retrieval and Processing

First all of the relevant packages had to be installed.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
import lxml

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

!pip install unicodecsv
import unicodecsv as csv
import time

!pip install geocoder
import geocoder # import geocoder

import re

print('Libraries imported.')

     |████████████████████████████████| 112kB 5.1MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 164kB/s eta 0:00:01     |███████████████████████████████▏| 5.6MB 164kB/s eta 0:00:01
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noa

### 1. Retrieve Real Estate Data

Real estate data was retrieved using the realtor.ca API. Data will be stored in a dictionary format. The following code was written using the information on these two github links: 

In [6]:
final_data = {} #initialize the library that will contain all of the listings. Using a library will prevent double entries of listings

url ="https://api2.realtor.ca/Listing.svc/PropertySearch_Post" #define the API

LandSizeRange = ["0-10", "10-50", "50-100", "100-320", "320-640", "640-1000", "1000-0"] #set land range to all possibilities
interval = 100000 #will search in 100,000 dollar intervals
intervals=60 #will search 60 intervals making the highest priced house that will be returned $6,000,000
max_results = 200 # the API has a limit of returning a maximum of 200 results at once

Prices = list((i * interval, i * interval + interval) for i in range(intervals)) #create price list

for PriceMin, PriceMax in Prices: #loop over each price in list 
    #create dictionary of values to search with
    search = {
        "CultureId": "1",
        "ApplicationId": "1",
        "RecordsPerPage": max_results,
        "MaximumResults": max_results,
        "PropertySearchTypeId": "1",   #property search type for residential
        "PriceMin": PriceMin,
        "PriceMax": PriceMax,
        "LandSizeRange": "0-0",
        "TransactionTypeId": "2",
        "StoreyRange": "0-0",
        "BedRange": "0-0",
        "BathRange": "0-0",
        "LongitudeMin":-114.2592 ,         #calgary boundaries
        "LongitudeMax":-113.8654 ,         #calgary boundaries
        "LatitudeMin": 50.8529,            #calgary boundaries
        "LatitudeMax": 51.2131,            #calgary boundaries
        "SortOrder": "A",
        "SortBy": "1",
        "viewState": "m",
        "Longitude": -114.0719,           #calgary center
        "Latitude": 51.0447,              #calgary center
        "ZoomLevel": "8",
        "City":"Calgary",
    }
    while True:  
        #if request is found then the while statement ends
        try: 
            r = requests.post(url, data=search)
            break
        #if the request is not found there is an error message and the while statement ends
        except:
            print(PriceMin, PriceMax)
            time.sleep(1)
            print('Connection Fail...')
            pass
    if r.ok:
        results = r.json()['Results']               #if there is a result it will be saved
        #since only 200 results can be retrieved at once this statement will break down that data so that all data is retrieved in 200 result batches
        if len(results) == max_results:     # if 200 results are returned  
            half = (PriceMax - PriceMin) / 2   #the interval spilt in two
            Prices.append([PriceMin, int(PriceMin + half)])  # new price intervals are appended to the price list
            Prices.append([int(PriceMin + half), PriceMax])
        
        #the desired results are extracted and saved in a dictionary
        for result in results:
            data = {
                'lng': result['Property']['Address']['Longitude'],
                'lat': result['Property']['Address']['Latitude'],
                'address': result['Property']['Address']['AddressText'],
                'building_type':result['Building'].get('Type','NA'),
                'price': result['Property']['Price'],
                'bathrooms': result['Building'].get('BathroomTotal', 0),
                'bedrooms': result['Building'].get('Bedrooms', 0),
                'InteriorSize': result['Building'].get('SizeInterior',0)
            }
            final_data[result['MlsNumber']] = data

print('Finished.', len(final_data),' results.')

Finished. 5735  results.


The library will then be converted to a CSV.

In [24]:
with open('mls.csv', 'wb') as f:
    writer = csv.DictWriter(f, fieldnames=next(iter(final_data.values())), dialect='excel')
    writer.writeheader()
    for row in final_data.values():
        writer.writerow(row)

Finally the CSV will be loaded into a dataframe

In [2]:
real_estate=pd.read_csv('mls.csv')
real_estate.head()

,lng,lat,address,property_type,building_type,price,bathrooms,bedrooms,InteriorSize
0,-113.941113,51.040125,"#211 6220 AV SE|Red Carpet, Calgary, Alberta ...",Single Family,Mobile Home,"$19,000",1,2 + 0,715 sqft
1,-113.937916,51.038809,"#278 6220 17 AV SE|Red Carpet, Calgary, Albert...",Single Family,Mobile Home,"$29,000",1,2 + 0,960 sqft
2,-114.214703,51.089611,"#16 3223 83 ST NW|Greenwood/Greenbriar, Calgar...",Single Family,Mobile Home,"$32,500",1,3 + 0,1249 sqft
3,-114.216046,51.089684,"#44 3223 83 ST NW|Bowness, Calgary, Alberta T3...",Single Family,Mobile Home,"$38,500",1,2 + 0,983 sqft
4,-114.217323,51.127602,"#166 99 ARBOUR LAKE RD NW|Arbour Lake, Calgary...",Single Family,Mobile Home,"$39,900",2,2 + 0,892 sqft


Extract the Neighbourbood from the Address

In [3]:
neighbourhood=[]
address=real_estate['address']
for i in range(len(address)):
    s = address[i]
    start = s.find('|') + 1
    end = s.find(', Calgary', start)
    neighbourhood.append(s[start:end])


Add the neighbourhood column into the real estate table. Since the dataframe has the latitude and longitude and now the neighbourhood has been extracted from the address, the address can be removed from the dataframe as it is no longer useful for the project's purposes.

In [4]:
real_estate['Neighbourhood']=neighbourhood #add in the Neighbourhood column
real_estate=real_estate.drop(['address'], axis=1)#remove address column 

#move the neighbourhood column to the front
fixed_columns = [real_estate.columns[-1]] + list(real_estate.columns[:-1])
real_estate = real_estate[fixed_columns]
pd.set_option('max_columns', None)

real_estate.head()

,Neighbourhood,lng,lat,property_type,building_type,price,bathrooms,bedrooms,InteriorSize
0,Red Carpet,-113.941113,51.040125,Single Family,Mobile Home,"$19,000",1,2 + 0,715 sqft
1,Red Carpet,-113.937916,51.038809,Single Family,Mobile Home,"$29,000",1,2 + 0,960 sqft
2,Greenwood/Greenbriar,-114.214703,51.089611,Single Family,Mobile Home,"$32,500",1,3 + 0,1249 sqft
3,Bowness,-114.216046,51.089684,Single Family,Mobile Home,"$38,500",1,2 + 0,983 sqft
4,Arbour Lake,-114.217323,51.127602,Single Family,Mobile Home,"$39,900",2,2 + 0,892 sqft


### 2. Retrieve Neighbourhood List and Coordinates

The Calgary neighbourhood list was retrieved from wikipedia and stored in a dataframe using the following code

In [5]:
#find list of neighbourhoods from wikipedia
res = requests.get("https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Calgary")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 

#Load into a dataframe
df = pd.read_html(str(table))
neigh_wiki=pd.DataFrame(df[0])

#Define header names
header_list=['Name','Quadrant','Sector','Ward','Type','2012_Population_Rank','2012_Population','2011_Population','%Change','Dwellings','Area','Population_Density']
neigh_wiki.columns=header_list

neigh_wiki.head()

,Name,Quadrant,Sector,Ward,Type,2012_Population_Rank,2012_Population,2011_Population,%Change,Dwellings,Area,Population_Density
0,Abbeydale,NE/SE,Northeast,10,Residential,82,5917.0,5700.0,3.8,2023.0,1.7,3480.6
1,Acadia,SE,South,9,Residential,27,10705.0,10615.0,0.8,5053.0,3.9,2744.9
2,Albert Park/Radisson Heights,SE,East,10,Residential,75,6234.0,6217.0,0.3,2709.0,2.5,2493.6
3,Altadore,SW,Centre,11,Residential,39,9116.0,8907.0,2.3,4486.0,2.9,3143.4
4,Alyth/Bonnybrook,SE,Centre,9,Industrial,208,16.0,17.0,−5.9,14.0,3.8,4.2


To clean up the rows anything that does not have a type of residential will be removed. There are also few neighbourhood names that need to be replaced due to wikipedia's formatting.

In [6]:
#remove rows that are not residential and reset the index
neigh_wiki.drop(neigh_wiki[neigh_wiki.Type != 'Residential'].index, inplace=True) #remove non-residential rows
neigh_wiki=neigh_wiki.reset_index(drop=True)

#replace some of the neighbourhood values that are in the wrong format
neigh_wiki['Name']=neigh_wiki.replace({'Name' : { 'CFB Currie':'Currie Barracks','CFB Lincoln Park PMQ':'Lincoln Park','Cityscape[2]':'Cityscape','Hotchkiss[3]':'Hotchkiss'}})



The only information needed from this dataframe now is the resulting neighbourhood list

In [7]:
#extract the 'Name' column into a list of neighbourhoods
neigh_list=neigh_wiki['Name'].tolist()

The information from wikipedia is old and outdated (from 2012) so there is a small list that needs to be appended.

In [8]:
#define list of neighbourhoods that need to be appended
add_neigh=['Livingston', 'University District', 'Yorkville','Carrington', 'Wolf Willow','Cornerstone','Belmont','Pine Creek','Garrison Green','Garrison Woods']
#append additional neighbourhoods
neigh_list.extend(add_neigh)

Google geocoder now requires payment for using the API and geolocator is unreliable, the approximate lat/long of each neighbourhood was found by averaging the lat/long values of the listings within each neighbourhood.

In [9]:
#create latitude and longitude lists that correspond with each neighbourhood
neigh_lat=[]
neigh_long=[]

for i in range(len(neigh_list)): #for each neighbourhood
    lat_total=0 
    long_total=0
    total=0
    for j in range(len(real_estate)): #search each listing 
        #if the listing neighbourhood matches the neighbourhood in the list add up the latidudes and longitudes and keep a count
        if neigh_list[i].lower().replace('.','')== real_estate['Neighbourhood'][j].lower().replace('.',''): 
            lat_total=lat_total+float(real_estate['lat'][j]) 
            long_total=long_total+float(real_estate['lng'][j])
            total=total+1
    #append the average latitude and longitude to the lat/long lists
    if total>0:
        neigh_lat.append(lat_total/total) #add average latitude to the neighbourhood latitude list
        neigh_long.append(long_total/total) #add average longitude to the neighbourhood latitude list
    else: # if there's no results for the neighbourhood in the list the lat/long will be defaulted to NA
        neigh_lat.append('NA')
        neigh_long.append('NA')
    
print('Approximate Neighbourhood Coordinates Found')
    

Approximate Neighbourhood Coordinates Found


Combine the Neighbourhoods and Coordinates into a Dataframe

In [10]:
neighbourhood_df=pd.DataFrame(list(zip(neigh_list,neigh_lat,neigh_long)),columns=['Neighbourhood','Latitude','Longitude'])
neighbourhood_df.drop(neighbourhood_df[neighbourhood_df.Latitude == 'NA'].index, inplace=True) #remove rows from neighbourhoods with no listings
neighbourhood_df=neighbourhood_df.reset_index(drop=True)
neighbourhood_df.head()

,Neighbourhood,Latitude,Longitude
0,Abbeydale,51.059,-113.926
1,Acadia,50.9735,-114.061
2,Albert Park/Radisson Heights,51.0404,-113.993
3,Altadore,51.0168,-114.104
4,Applewood Park,51.0422,-113.93


Using the neighbourhood list rows can be removed from the real estate table that do not have a valid neighbourhood listed by comparing the the neighbourhood column to the list that was scraped off wikipedia.

In [11]:
for i in range(len(real_estate)): #search each real estate listing
    match=False
    for j in range(len(neigh_list)): #search each neighbourhood
        #if the neighbourhood in the real estate list matches the neighbourhood list the condition will change to true 
        if neigh_list[j].lower().replace('.','')== real_estate['Neighbourhood'][i].lower().replace('.',''):
            match=True
    #if no match is found then the listing is removed
    if match==False:
        real_estate.drop([i], inplace=True)
print('matching finished')
        

matching finished


In [12]:
#reindex after rows have been removed
real_estate=real_estate.reset_index(drop=True)

Now there are two dataframes: one that has a list of neighbourhoods and the respective coordinates and the other has all the property listings with their relevant information.

### 3. Retrieve Neighbourhood Venue Information

Neighbourhood venue information will be retrieved using the Four Square API. Venues will be returned within a radius of 750m of the neighbourhood center.

In [97]:
# four square credentials
CLIENT_ID = 'H2MLZR40MMHMP14IHCXMQNRGKJMHZAZFZIQLRKNAI431KU3Q' # your Foursquare ID
CLIENT_SECRET = 'QE4APSO5LDTUMPFLZBXULPGDLIOYRJXXY04W10EUFNEQP2PI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [98]:
#define the function to get nearby venues for each neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
#Use the function getNeaarbyVenues to retrieve venue venue information
LIMIT=100
calgary_venues = getNearbyVenues(names=neighbourhood_df['Neighbourhood'],
                                   latitudes=neighbourhood_df['Latitude'],
                                   longitudes= neighbourhood_df['Longitude']
                                  )
print('Venue Search Complete')

Venue Search Complete


In [102]:
#write to csv due to limited amount of data retrieval per day
calgary_venues.to_csv('Calgary_Venues.csv', index=False)

#check size of venue dataframe
print(calgary_venues.shape)

calgary_venues.head()

(2878, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abbeydale,51.059006,-113.925914,Subway,51.059239,-113.934423,Sandwich Place
1,Abbeydale,51.059006,-113.925914,Vanity Fitness,51.062982,-113.926238,Health & Beauty Service
2,Abbeydale,51.059006,-113.925914,Mac's,51.059376,-113.934425,Convenience Store
3,Abbeydale,51.059006,-113.925914,roadside pub,51.059277,-113.934529,Wings Joint
4,Abbeydale,51.059006,-113.925914,Magic Touch Stone Ltd,51.065379,-113.927304,Construction & Landscaping


In [103]:
print('There are {} uniques categories.'.format(len(calgary_venues['Venue Category'].unique()))) 

There are 256 uniques categories.


## **Methodology**

Neighbourhoods will be clustered based on both the venue data and the real estate. For both of these sets of data, dummy values will have to be obtained for the columns containing data to be used in the clustering. The dummies allow the data to be represented numerically and ranked so that the k-means clustering algorithm can be applied. 

The analysis of the data will be using the following steps:
1. Create a map of Calgary neighbourhoods to visualize the neighbourhood locations
2. Get dummies, rank, and then cluster the venue data 
3. Get dummies, rank, and then cluster the real estate data
4. Combine venue data and real estate dummy data and run clustering

### 1. Calgary Neighbourhood Map

Using folium a map of Calgary and the neighbourhoods with residential sales listings is created to visualize neighbourhood locations. This can also help to make sure the neighbourhood coordinates appear where they should.

In [14]:
#find geographical coordinates of calgary using geolocator
city = 'Calgary, AB'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(city)
city_lat = location.latitude
city_long = location.longitude
print('The geograpical coordinate of Calgary are {}, {}.'.format(city_lat, city_long))

The geograpical coordinate of Calgary are 51.0534234, -114.0625892.


In [15]:
#create a map of Calgary using folium
map_calgary = folium.Map(location=[city_lat, city_long], zoom_start=10)

# add neighbourhood markers to map
for lat, lng, neighbourhood in zip(neighbourhood_df['Latitude'], neighbourhood_df['Longitude'], neighbourhood_df['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

### 2. Clustering Based on Neighbourhood Venues

K-means clustering will be performed using the Venue Category column from the venue dataframe. This is only column that dummy values will be obtained for in the venue dataframe.

A venue count column will be added to keep track of number of venues for each neighbourhood

In [16]:
venue_count=calgary_venues.groupby('Neighbourhood').count().reset_index()
venue_count=venue_count.rename(columns={'Venue':'Venue_Count'})
venue_count.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue_Count,Venue Latitude,Venue Longitude,Venue Category
0,Abbeydale,5,5,5,5,5,5
1,Acadia,21,21,21,21,21,21
2,Albert Park/Radisson Heights,13,13,13,13,13,13
3,Altadore,9,9,9,9,9,9
4,Applewood Park,5,5,5,5,5,5


In [17]:
venue_count=venue_count.iloc[:,[0,3]] #save the dataframe so that it only contains the neighbourhood and venue count
venue_count=venue_count
venue_count.head()

,Neighbourhood,Venue_Count
0,Abbeydale,5
1,Acadia,21
2,Albert Park/Radisson Heights,13
3,Altadore,9
4,Applewood Park,5


In [ ]:
#breakdown the counts into groups so that clustering is easier

for i in range(len(venue_count)): #check and replace each entry with a categorized count
    if int(venue_count['Venue_Count'][i])>50:
        venue_count['Venue_Count'][i]='50+'
    elif int(venue_count['Venue_Count'][i])>=25 and int(venue_count['Venue_Count'][i])<50:
        venue_count['Venue_Count'][i]='25-50'
    elif int(venue_count['Venue_Count'][i])<25:
        venue_count['Venue_Count'][i]='<25'

These counts will be inserted back into the calgary venue dataframe.

In [19]:
calgary_venues = pd.merge(calgary_venues,venue_count, on='Neighbourhood')
calgary_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue_Count
0,Abbeydale,51.059006,-113.925914,Subway,51.059239,-113.934423,Sandwich Place,<25
1,Abbeydale,51.059006,-113.925914,Vanity Fitness,51.062982,-113.926238,Health & Beauty Service,<25
2,Abbeydale,51.059006,-113.925914,Mac's,51.059376,-113.934425,Convenience Store,<25
3,Abbeydale,51.059006,-113.925914,roadside pub,51.059277,-113.934529,Wings Joint,<25
4,Abbeydale,51.059006,-113.925914,Magic Touch Stone Ltd,51.065379,-113.927304,Construction & Landscaping,<25


A dummy dataframe is first created for the Venues Category column

In [22]:
# Use one hot coding to get dummy values for the venues category
calgary_onehot_cat = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe of dummy values
calgary_onehot_cat['Neighbourhood'] = calgary_venues['Neighbourhood']

# move neighbourhood column to the first column
fixed_columns = [calgary_onehot_cat.columns[-1]] + list(calgary_onehot_cat.columns[:-1])
calgary_onehot_cat = calgary_onehot_cat[fixed_columns]
pd.set_option('max_columns', None)
calgary_onehot_cat.head()

,Neighbourhood,ATM,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Communications Building,College Gym,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paintball Field,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Professional & Other Places,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel & Transport,Tree,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abbeydale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

A dummy dataframe is then created for the venue count column

In [20]:
# Use one hot coding to get dummy values for the venue count
calgary_onehot_count= pd.get_dummies(calgary_venues[['Venue_Count']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe of dummy values
calgary_onehot_count['Neighbourhood'] = calgary_venues['Neighbourhood']

# move neighbourhood column to the first column
fixed_columns = [calgary_onehot_count.columns[-1]] + list(calgary_onehot_count.columns[:-1])
calgary_onehot_count = calgary_onehot_count[fixed_columns]
pd.set_option('max_columns', None)
calgary_onehot_count.head()

,Neighbourhood,25-50,50+,<25
0,Abbeydale,0,0,1
1,Abbeydale,0,0,1
2,Abbeydale,0,0,1
3,Abbeydale,0,0,1
4,Abbeydale,0,0,1


Grouping is performed for each dummy dataframe

In [23]:
#group the venue categories dummy dataframe so that there is only one row per neighbourhood
calgary_venues.groupby('Neighbourhood').count()
calgary_grouped_cat = calgary_onehot_cat.groupby('Neighbourhood').mean().reset_index() #group by each neighbourhood and average the values
calgary_grouped_cat.head()

,Neighbourhood,ATM,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Communications Building,College Gym,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paintball Field,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Professional & Other Places,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel & Transport,Tree,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Abbeydale,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,

In [24]:
#group the venue categories dummy dataframe so that there is only one row per neighbourhood
calgary_venues.groupby('Neighbourhood').count()
calgary_grouped_count = calgary_onehot_count.groupby('Neighbourhood').mean().reset_index() #group by each neighbourhood and average the values
calgary_grouped_count.head()

,Neighbourhood,25-50,50+,<25
0,Abbeydale,0,0,1
1,Acadia,0,0,1
2,Albert Park/Radisson Heights,0,0,1
3,Altadore,0,0,1
4,Applewood Park,0,0,1


The Venue Category and Venue Count grouped dataframes are then merged

In [25]:
#merge the venue and count grouped data
calgary_grouped = pd.merge(calgary_grouped_cat,calgary_grouped_count, on='Neighbourhood')
calgary_grouped.head()

,Neighbourhood,ATM,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Communications Building,College Gym,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paintball Field,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Professional & Other Places,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Travel & Transport,Tree,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,25-50,50+,<25
0,Abbeydale,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Next, using the the two grouped values dataframes ,tables are created for the top 5 venues in each neighbourhood and a table of the venue count.

In [26]:
#create function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
#use function to find top 5 venues for each neighbourhood and put this in a dataframe
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe with neighbourhood and top 5 venues
neighbourhoods_venues_sorted_cat = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted_cat['Neighbourhood'] = calgary_grouped_cat['Neighbourhood']

for ind in np.arange(calgary_grouped_cat.shape[0]):
    neighbourhoods_venues_sorted_cat.iloc[ind, 1:] = return_most_common_venues(calgary_grouped_cat.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted_cat.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abbeydale,Wings Joint,Health & Beauty Service,Sandwich Place,Convenience Store,Construction & Landscaping
1,Acadia,Sandwich Place,Pub,Fast Food Restaurant,Coffee Shop,Yoga Studio
2,Albert Park/Radisson Heights,Grocery Store,Indian Restaurant,Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,Altadore,Liquor Store,Coffee Shop,Pub,Brewery,Greek Restaurant
4,Applewood Park,Home Service,Coffee Shop,Park,Liquor Store,Food & Drink Shop


In [28]:
#use function to find the number of venues close to the neighbourhood
num_top_venues = 1


# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append(' Most Common Venue Count'.format(ind+1, indicators[ind]))

# create a new dataframe with neighbourhood and top 5 venues
neighbourhoods_venues_sorted_count = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted_count['Neighbourhood'] = calgary_grouped_count['Neighbourhood']

for ind in np.arange(calgary_grouped_cat.shape[0]):
    neighbourhoods_venues_sorted_count.iloc[ind, 1:] = return_most_common_venues(calgary_grouped_count.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted_count.head()

,Neighbourhood,Most Common Venue Count
0,Abbeydale,<25
1,Acadia,<25
2,Albert Park/Radisson Heights,<25
3,Altadore,<25
4,Applewood Park,<25


Merge the Venue Category and Venue Count sorted dataframes of most common values

In [29]:
#merge the venue and count sorted data
neighbourhood_venues_sorted = pd.merge(neighbourhoods_venues_sorted_cat,neighbourhoods_venues_sorted_count, on='Neighbourhood')
neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
0,Abbeydale,Wings Joint,Health & Beauty Service,Sandwich Place,Convenience Store,Construction & Landscaping,<25
1,Acadia,Sandwich Place,Pub,Fast Food Restaurant,Coffee Shop,Yoga Studio,<25
2,Albert Park/Radisson Heights,Grocery Store,Indian Restaurant,Restaurant,Fried Chicken Joint,Fast Food Restaurant,<25
3,Altadore,Liquor Store,Coffee Shop,Pub,Brewery,Greek Restaurant,<25
4,Applewood Park,Home Service,Coffee Shop,Park,Liquor Store,Food & Drink Shop,<25


Finally using the dataframe of the top 5 venues for each neighbourhood and the venue count, the neighbourhoods are clustered.

In [30]:
# set number of clusters
kclusters = 5

#define the data to use for clustering
calgary_grouped_clustering = calgary_grouped.drop(columns=['Neighbourhood'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(calgary_grouped_clustering)

In [31]:
# add clustering labels
neighbourhood_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

calgary_merged = neighbourhood_df

# merge dataframes
calgary_merged = calgary_merged.join(neighbourhood_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

calgary_merged.head() 

,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
0,Abbeydale,51.059,-113.926,0.0,Wings Joint,Health & Beauty Service,Sandwich Place,Convenience Store,Construction & Landscaping,<25
1,Acadia,50.9735,-114.061,2.0,Sandwich Place,Pub,Fast Food Restaurant,Coffee Shop,Yoga Studio,<25
2,Albert Park/Radisson Heights,51.0404,-113.993,2.0,Grocery Store,Indian Restaurant,Restaurant,Fried Chicken Joint,Fast Food Restaurant,<25
3,Altadore,51.0168,-114.104,2.0,Liquor Store,Coffee Shop,Pub,Brewery,Greek Restaurant,<25
4,Applewood Park,51.0422,-113.93,2.0,Home Service,Coffee Shop,Park,Liquor Store,Food & Drink Shop,<25


In [32]:
#remove rows with Cluster labels that are not numeric
calgary_merged=calgary_merged.dropna()
calgary_merged=calgary_merged.reset_index(drop=True)

In [33]:
# create map to visualize results
map_clusters = folium.Map(location=[city_lat, city_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(calgary_merged['Latitude'], calgary_merged['Longitude'], calgary_merged['Neighbourhood'], calgary_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clustering Based on House Attributes

K-means clustering will be performed using data from several columns from the real estate data. 
The data of interest is the building type, the price, the number of bedrooms, the number of bathrooms, and the interior square footage.

First the data is going to be broken down so that it is easier to cluster. Price and interior size will be grouped into brackets and bedrooms will be reformatted into a single number.

In [34]:
#format price and interior size so that both can be evaluated as an integer
real_estate['price']=real_estate['price'].str.replace( '$','')
real_estate['price']=real_estate['price'].str.replace(',' , '')
real_estate['InteriorSize']=real_estate['InteriorSize'].str.replace('sqft','')


for i in range(len(real_estate)): #search each listing
    #categorize and replace prices
    if int(real_estate['price'][i])<=250000:
        real_estate['price'][i]='<=$250,000'
    elif int(real_estate['price'][i]) >250000 and int(real_estate['price'][i])<=500000:
        real_estate['price'][i]='$250,000-$500,000'
    elif int(real_estate['price'][i])>500000 and int(real_estate['price'][i])<=750000:
        real_estate['price'][i]='$500,000-$750,000'
    elif int(real_estate['price'][i]) >750000 and int(real_estate['price'][i])<=1000000:
        real_estate['price'][i]='$750,000-$1,000,000'
    elif int(real_estate['price'][i]) >1000000 and int(real_estate['price'][i])<=2000000:
        real_estate['price'][i]='$1,000,000-$2,000,000'
    elif int(real_estate['price'][i]) >2000000 and int(real_estate['price'][i])<=3000000:
        real_estate['price'][i]='$2,000,000-$3,000,000'
    elif int(real_estate['price'][i]) >3000000:
        real_estate['price'][i]='$3,000,000+'
    #categorize and replace interior size
    if int(real_estate['InteriorSize'][i]) <=500:
        real_estate['InteriorSize'][i]='<= 500 sqft'
    elif int(real_estate['InteriorSize'][i])  >500 and int(real_estate['InteriorSize'][i]) <=1000:
        real_estate['InteriorSize'][i]='500-1000 sqft'
    elif int(real_estate['InteriorSize'][i])  >1000 and int(real_estate['InteriorSize'][i]) <=2000:
        real_estate['InteriorSize'][i]='1000-2000 sqft'
    elif int(real_estate['InteriorSize'][i]) >2000:
        real_estate['InteriorSize'][i]='2000+ sqft'
    #reformat bedrooms so that they are a single number instead of 'number + number'
    real_estate['bedrooms'][i]= int(real_estate['bedrooms'][i][0])+int(real_estate['bedrooms'][i][-1])

In [35]:
#add bathroom string onto each bathroom entry (this will make the data easier to understand after dummy dataframe is created)
real_estate['bathrooms'] = real_estate['bathrooms'].astype(str)
real_estate['bathrooms'] =real_estate['bathrooms']+' bathrooms'
#add bedroom string onto each bedroom entry (this will make the data easier to understand after dummy dataframe is created)
real_estate['bedrooms'] =real_estate['bedrooms'].astype(str)+' bedrooms'

real_estate.head()

,Neighbourhood,lng,lat,property_type,building_type,price,bathrooms,bedrooms,InteriorSize
0,Red Carpet,-113.941113,51.040125,Single Family,Mobile Home,"<=$250,000",1 bathrooms,2 bedrooms,500-1000 sqft
1,Red Carpet,-113.937916,51.038809,Single Family,Mobile Home,"<=$250,000",1 bathrooms,2 bedrooms,500-1000 sqft
2,Greenwood/Greenbriar,-114.214703,51.089611,Single Family,Mobile Home,"<=$250,000",1 bathrooms,3 bedrooms,1000-2000 sqft
3,Bowness,-114.216046,51.089684,Single Family,Mobile Home,"<=$250,000",1 bathrooms,2 bedrooms,500-1000 sqft
4,Arbour Lake,-114.217323,51.127602,Single Family,Mobile Home,"<=$250,000",2 bathrooms,2 bedrooms,500-1000 sqft


The next step is to create dummy dataframes for the relevant real estate attributes. Then, similar to the analysis of the venue data, each attribute that a dummy dataframe was created for will be grouped based on neighbourhood and the top 2 categories for each attribute will be retrieved. Since there are several columns to perform this on, both of these steps take place in the same loop.

In [36]:
#create function to return most common attributes
def return_most_common_attributes(row, num_top):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top]

In [37]:
#create a list of attributes that dummies need to be created for and that the top 2 categories need to be found for
attributes=['building_type','price','bedrooms','bathrooms','InteriorSize']

#need to set a condition that will intialize a merged dataframe only on the first run of the loop
condition=False

#define top number of categories returned
num_top = 2

#set indicator list for column names
indicators = ['st', 'nd', 'rd']

for item in attributes: #for each attribute 
    
    #a dummy matrix is created
    onehot = pd.get_dummies(real_estate[[item]], prefix="", prefix_sep="") 
    onehot['Neighbourhood'] = real_estate['Neighbourhood']
    fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
    onehot = onehot[fixed_columns]
    pd.set_option('max_columns', None)

    #group by neighbourhood
    grouped = onehot.groupby('Neighbourhood').mean().reset_index() #group by each neighbourhood
    
    #intialize columns 
    columns = ['Neighbourhood']
    
    #this condition will initialize the creation of a merged matrix and prevent it from being overwritten in subsequent loops
    if condition==False:
        sorted_merge_df = pd.DataFrame(columns=columns)
        sorted_merge_df['Neighbourhood'] = grouped['Neighbourhood']
        grouped_merge_df = pd.DataFrame(columns=columns)
        grouped_merge_df['Neighbourhood'] = grouped['Neighbourhood']
        condition=True 
   
    # create columns according to number of top attribute categories
    for ind in np.arange(num_top):
        try:
            columns.append(('{}{} Most Common '+ item).format(ind+1, indicators[ind]))
        except:
            columns.append(('{}th Most Common ' +item).format(ind+1))

    # create a new dataframe for top categories
   
    sorted = pd.DataFrame(columns=columns)
    sorted['Neighbourhood'] = grouped['Neighbourhood']

    for ind in np.arange(grouped.shape[0]):
        sorted.iloc[ind, 1:] = return_most_common_attributes(grouped.iloc[ind, :], num_top)

    # merge the attribute results with the existing dataframe so that all results are stored in these    
    sorted_merge_df=pd.merge(sorted_merge_df, sorted, on='Neighbourhood')
    grouped_merge_df=pd.merge(grouped_merge_df, grouped, on='Neighbourhood')


In [38]:
#view results
grouped_merge_df.head()

,Neighbourhood,Apartment,Duplex,Fourplex,House,Mobile Home,Row / Townhouse,"$1,000,000-$2,000,000","$2,000,000-$3,000,000","$250,000-$500,000","$3,000,000+","$500,000-$750,000","$750,000-$1,000,000","<=$250,000",0 bedrooms,1 bedrooms,10 bedrooms,11 bedrooms,12 bedrooms,2 bedrooms,3 bedrooms,4 bedrooms,5 bedrooms,6 bedrooms,7 bedrooms,8 bedrooms,0 bathrooms,1 bathrooms,17 bathrooms,2 bathrooms,3 bathrooms,4 bathrooms,5 bathrooms,6 bathrooms,7 bathrooms,8 bathrooms,9 bathrooms,1000-2000 sqft,2000+ sqft,500-1000 sqft,<= 500 sqft
0,Abbeydale,0.000000,0.235294,0.0,0.470588,0.235294,0.058824,0.000000,0.000000,0.647059,0.0,0.000000,0.000000,0.352941,0.0,0.000000,0.0,0.0,0.0,0.000000,0.588235,0.294118,0.117647,0.0,0.0,0.0,0.0,0.117647,0.0,0.705882,0.176471,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.705882,0.000000,0.294118,0.000000
1,Acadia,0.333333,0.055556,0.0,0.527778,0.000000,0.083333,0.000000,0.000000,0.361111,0.0,0.305556,0.000000,0.333333,0.0,0.194444,0.0,0.0,0.0,0.166667,0.194444,0.416667,0.027778,0.0,0.0,0.0,0.0,0.277778,0.0,0.388889,0.333333,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.638889,0.000000,0.361111,0.000000
2,Albert Park/Radisson Heights,0.625000,0.250000,0.0,0.125000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.125000,0.000000,0.625000,0.0,0.000000,0.0,0.0,0.0,0.625000,0.250000,0.125000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.750000,0.125000,0.125000,0.00000,0.0,0.000000,0.0,0.0,0.125000,0.000000,0.875000,0.000000
3,Altadore,0.076923,0.241758,0.0,0.439560,0.000000,0.241758,0.241758,0.032967,0.120879,0.0,0.241758,0.307692,0.054945,0.0,0.076923,0.0,0.0,0.0,0.109890,0.186813,0.505495,0.120879,0.0,0.0,0.0,0.0,0.098901,0.0,0.142857,0.142857,0.494505,0.10989,0.0,0.010989,0.0,0.0,0.549451,0.296703,0.142857,0.010989
4,Applewood Park,0.200000,0.100000,0.0,0.600000,0.000000,0.100000,0.000000,0.000000,0.700000,0.0,0.000000,0.000000,0.300000,0.0,0.100000,0.0,0.0,0.0,0.200000,0.400000,0.200000,0.000000,0.1,0.0,0.0,0.0,0.100000,0.1,0.500000,0.200000,0.100000,0.00000,0.0,0.000000,0.0,0.0,0.800000,0.000000,0.200000,0.000000


In [39]:
#view results
sorted_merge_df.head()

,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize
0,Abbeydale,House,Mobile Home,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft
1,Acadia,House,Apartment,"$250,000-$500,000","<=$250,000",4 bedrooms,3 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft
2,Albert Park/Radisson Heights,Apartment,Duplex,"<=$250,000","$250,000-$500,000",2 bedrooms,3 bedrooms,2 bathrooms,4 bathrooms,500-1000 sqft,1000-2000 sqft
3,Altadore,House,Row / Townhouse,"$750,000-$1,000,000","$500,000-$750,000",4 bedrooms,3 bedrooms,4 bathrooms,3 bathrooms,1000-2000 sqft,2000+ sqft
4,Applewood Park,House,Apartment,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft


The real estate data from the merged top attribute categories is now clustered using K-means

In [40]:
# set number of clusters
kclusters = 5

#define data to be run in K-means
re_grouped_clustering = grouped_merge_df.drop(columns=['Neighbourhood'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(re_grouped_clustering)
 

In [41]:
# add clustering labels
sorted_merge_df.insert(0, 'Cluster_Labels', kmeans.labels_)

#created a dataframe consisting of the top attribute categories and the neighbourhood coordinates
all_merged = neighbourhood_df
all_merged = all_merged.join(sorted_merge_df.set_index('Neighbourhood'), on='Neighbourhood')

all_merged.head() 

,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize
0,Abbeydale,51.059,-113.926,1.0,House,Mobile Home,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft
1,Acadia,50.9735,-114.061,1.0,House,Apartment,"$250,000-$500,000","<=$250,000",4 bedrooms,3 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft
2,Albert Park/Radisson Heights,51.0404,-113.993,2.0,Apartment,Duplex,"<=$250,000","$250,000-$500,000",2 bedrooms,3 bedrooms,2 bathrooms,4 bathrooms,500-1000 sqft,1000-2000 sqft
3,Altadore,51.0168,-114.104,3.0,House,Row / Townhouse,"$750,000-$1,000,000","$500,000-$750,000",4 bedrooms,3 bedrooms,4 bathrooms,3 bathrooms,1000-2000 sqft,2000+ sqft
4,Applewood Park,51.0422,-113.93,1.0,House,Apartment,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft


In [42]:
#remove rows with Cluster labels that are not numeric and reset index
all_merged=all_merged.dropna()
all_merged=all_merged.reset_index(drop=True)

In [43]:
# create map to visualize results
map_clusters = folium.Map(location=[city_lat, city_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighbourhood'], all_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Clustering Based on Venue Data and House Attributes

The real estate data and the venue data sorted and grouped dataframes will be merged to perform clustering using all of the data

In [46]:
#merge the real estate and venue sorted data
finished_merge_sorted=pd.merge(sorted_merge_df, neighbourhood_venues_sorted, on='Neighbourhood')
finished_merge_sorted=finished_merge_sorted.drop(['Cluster_Labels_x','Cluster_Labels_y'],axis=1)
finished_merge_sorted.head()

,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
0,Abbeydale,House,Mobile Home,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Wings Joint,Health & Beauty Service,Sandwich Place,Convenience Store,Construction & Landscaping,<25
1,Acadia,House,Apartment,"$250,000-$500,000","<=$250,000",4 bedrooms,3 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Sandwich Place,Pub,Fast Food Restaurant,Coffee Shop,Yoga Studio,<25
2,Albert Park/Radisson Heights,Apartment,Duplex,"<=$250,000","$250,000-$500,000",2 bedrooms,3 bedrooms,2 bathrooms,4 bathrooms,500-1000 sqft,1000-2000 sqft,Grocery Store,Indian Restaurant,Restaurant,Fried Chicken Joint,Fast Food Restaurant,<25
3,Altadore,House,Row / Townhouse,"$750,000-$1,000,000","$500,000-$750,000",4 bedrooms,3 bedrooms,4 bathrooms,3 bathrooms,1000-2000 sqft,2000+ sqft,Liquor Store,Coffee Shop,Pub,Brewery,Greek Restaurant,<25
4,Applewood Park,House,Apartment,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Home Service,Coffee Shop,Park,Liquor Store,Food & Drink Shop,<25


In [47]:
#merge the real estate and venue grouped data
finished_merge_grouped=pd.merge(grouped_merge_df, calgary_grouped, on='Neighbourhood')
finished_merge_grouped.head()

,Neighbourhood,Apartment,Duplex,Fourplex,House,Mobile Home,Row / Townhouse,"$1,000,000-$2,000,000","$2,000,000-$3,000,000","$250,000-$500,000","$3,000,000+","$500,000-$750,000","$750,000-$1,000,000","<=$250,000",0 bedrooms,1 bedrooms,10 bedrooms,11 bedrooms,12 bedrooms,2 bedrooms,3 bedrooms,4 bedrooms,5 bedrooms,6 bedrooms,7 bedrooms,8 bedrooms,0 bathrooms,1 bathrooms,17 bathrooms,2 bathrooms,3 bathrooms,4 bathrooms,5 bathrooms,6 bathrooms,7 bathrooms,8 bathrooms,9 bathrooms,1000-2000 sqft,2000+ sqft,500-1000 sqft,<= 500 sqft,ATM,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Communications Building,College Gym,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Country Dance Club,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Event Space,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Shop,Gym,Gym / Fitness Center,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hockey Rink,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Inn,Intersection,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paintball Field,Paper / Office Supplies Store,Park,Peking Duck Restaurant,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Professional & Other Places,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Lodge,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Student Center,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,

In [48]:
# set number of clusters
kclusters = 5

#define data to be run in K-means
finished_grouped_clustering = finished_merge_grouped.drop(columns=['Neighbourhood'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(finished_grouped_clustering)
 

In [49]:
# add clustering labels
finished_merge_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

final_merged = neighbourhood_df

# merge data and add latitude/longitude for each neighborhood
final_merged = final_merged.join(finished_merge_sorted.set_index('Neighbourhood'), on='Neighbourhood')

final_merged.head() 

,Neighbourhood,Latitude,Longitude,Cluster_Labels,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
0,Abbeydale,51.059,-113.926,2.0,House,Mobile Home,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Wings Joint,Health & Beauty Service,Sandwich Place,Convenience Store,Construction & Landscaping,<25
1,Acadia,50.9735,-114.061,2.0,House,Apartment,"$250,000-$500,000","<=$250,000",4 bedrooms,3 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Sandwich Place,Pub,Fast Food Restaurant,Coffee Shop,Yoga Studio,<25
2,Albert Park/Radisson Heights,51.0404,-113.993,2.0,Apartment,Duplex,"<=$250,000","$250,000-$500,000",2 bedrooms,3 bedrooms,2 bathrooms,4 bathrooms,500-1000 sqft,1000-2000 sqft,Grocery Store,Indian Restaurant,Restaurant,Fried Chicken Joint,Fast Food Restaurant,<25
3,Altadore,51.0168,-114.104,0.0,House,Row / Townhouse,"$750,000-$1,000,000","$500,000-$750,000",4 bedrooms,3 bedrooms,4 bathrooms,3 bathrooms,1000-2000 sqft,2000+ sqft,Liquor Store,Coffee Shop,Pub,Brewery,Greek Restaurant,<25
4,Applewood Park,51.0422,-113.93,2.0,House,Apartment,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Home Service,Coffee Shop,Park,Liquor Store,Food & Drink Shop,<25


In [50]:
#remove rows with Cluster labels that are not numeric and reset index
final_merged=final_merged.dropna()
final_merged=final_merged.reset_index(drop=True)

In [51]:
# create map to visualize results
map_clusters = folium.Map(location=[city_lat, city_long], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(final_merged['Latitude'], final_merged['Longitude'], final_merged['Neighbourhood'], final_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## **Results**

Here each neighbourhood cluster will be examined to help form the 5 different types of buying criteria

**Cluster 0**

Summary: Buyers interested in this cluster would be looking for a large and more expensive house or townhouse. Common prices are over 500,000 dollars and include neighbourhoods with average house prices over one million. The homes in this cluster are usually are over 1000 sqft with many over options 2000 sqft and have at least 4 bedrooms and at least 4 bathrooms. Looking at the map most of these neighbourhoods closer to the inner city. The neighbourhoods in this cluster will have lower accessibility to venues as the venue count is commonly <25.

In [173]:
#cluster 0
final_merged.loc[final_merged['Cluster_Labels'] == 0, final_merged.columns[[0] + list(range(4, final_merged.shape[1]))]]


,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
3,Altadore,House,Row / Townhouse,"$750,000-$1,000,000","$500,000-$750,000",4 bedrooms,3 bedrooms,4 bathrooms,3 bathrooms,1000-2000 sqft,2000+ sqft,Liquor Store,Coffee Shop,Pub,Brewery,Greek Restaurant,<25
8,Banff Trail,Duplex,House,"$500,000-$750,000","$750,000-$1,000,000",4 bedrooms,3 bedrooms,4 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,Hotel,Coffee Shop,Ice Cream Shop,Gourmet Shop,Liquor Store,<25
10,Bayview,House,Duplex,"$750,000-$1,000,000","$1,000,000-$2,000,000",5 bedrooms,2 bedrooms,4 bathrooms,9 bathrooms,2000+ sqft,<= 500 sqft,Pub,Other Great Outdoors,Park,Grocery Store,Breakfast Spot,<25
12,Bel-Aire,House,Row / Townhouse,"$3,000,000+","$1,000,000-$2,000,000",4 bedrooms,6 bedrooms,3 bathrooms,8 bathrooms,2000+ sqft,1000-2000 sqft,Clothing Store,Breakfast Spot,Golf Course,Home Service,Food Truck,<25
20,Britannia,House,Apartment,"$2,000,000-$3,000,000","$750,000-$1,000,000",5 bedrooms,4 bedrooms,3 bathrooms,8 bathrooms,2000+ sqft,1000-2000 sqft,Coffee Shop,Bank,Shopping Mall,Restaurant,Food & Drink Shop,<25
21,Cambrian Heights,House,Duplex,"$500,000-$750,000","$750,000-$1,000,000",4 bedrooms,5 bedrooms,2 bathrooms,4 bathrooms,1000-2000 sqft,2000+ sqft,Indian Restaurant,Pharmacy,Grocery Store,Liquor Store,Coffee Shop,<25
23,Capitol Hill,House,Duplex,"$500,000-$750,000","$750,000-$1,000,000",4 bedrooms,3 bedrooms,4 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,Coffee Shop,Pub,Bank,Bookstore,Sporting Goods Shop,<25
26,Currie Barracks,Row / Townhouse,House,"$500,000-$750,000","$1,000,000-$2,000,000",3 bedrooms,4 bedrooms,4 bathrooms,3 bathrooms,2000+ sqft,1000-2000 sqft,Coffee Shop,Concert Hall,Music Venue,Gym,Food Court,<25
29,Charleswood,House,Row / Townhouse,"$500,000-$750,000","$750,000-$1,000,000",4 bedrooms,5 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,2000+ sqft,Elementary School,Pizza Place,Yoga Studio,Food Court,Filipino Restaurant,<25
32,Christie Park,House,Row / Townhouse,"$750,000-$1,000,000","$250,000-$500,000",4 bedrooms,2 bedrooms,4 bathrooms,2 bathrooms,2000+ sqft,1000-2000 sqft,Sandwich Place,Train Station,Pub,Coffee Shop,Japanese Restaurant,<25


**Cluster 1**

Summary: Buyers interested in this cluster would be looking an apartment or townhouse. Prices are commonly under 500,000 but there are a range of price options within this cluster. They are commonly under 2000 sqft with many options under 1000 sqft with less than 4 bedrooms and bathrooms. These neighbourhoods have a venue count of 25-50 meaning these neighbourhoods have good venue accessibility and by looking at the map are just outside the city core.

In [174]:
#cluster 1
final_merged.loc[final_merged['Cluster_Labels'] == 1, final_merged.columns[[0] + list(range(4, final_merged.shape[1]))]]

,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
6,Aspen Woods,House,Row / Townhouse,"$1,000,000-$2,000,000","$250,000-$500,000",3 bedrooms,4 bedrooms,4 bathrooms,3 bathrooms,2000+ sqft,1000-2000 sqft,Coffee Shop,Sandwich Place,Restaurant,Asian Restaurant,Yoga Studio,25-50
18,Bridgeland/Riverside,Apartment,House,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,1000-2000 sqft,Italian Restaurant,Coffee Shop,Sandwich Place,Sushi Restaurant,Pizza Place,25-50
27,Lincoln Park,Apartment,Row / Townhouse,"<=$250,000","$250,000-$500,000",1 bedrooms,3 bedrooms,1 bathrooms,3 bathrooms,500-1000 sqft,1000-2000 sqft,Coffee Shop,Pizza Place,Food Court,Sandwich Place,Bar,25-50
30,Chinatown,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",1 bedrooms,2 bedrooms,1 bathrooms,2 bathrooms,500-1000 sqft,1000-2000 sqft,Coffee Shop,Restaurant,Hotel,Park,Café,25-50
42,Country Hills Village,Apartment,Row / Townhouse,"<=$250,000","$250,000-$500,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,1000-2000 sqft,Coffee Shop,Fast Food Restaurant,Paper / Office Supplies Store,Juice Bar,Grocery Store,25-50
63,Erlton,Apartment,Row / Townhouse,"$250,000-$500,000","$500,000-$750,000",2 bedrooms,3 bedrooms,2 bathrooms,5 bathrooms,1000-2000 sqft,500-1000 sqft,Restaurant,Coffee Shop,Breakfast Spot,Gym / Fitness Center,Park,25-50
85,Inglewood,Apartment,House,"$250,000-$500,000","$500,000-$750,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,1000-2000 sqft,Exhibit,Diner,Gas Station,Concert Hall,Music Venue,25-50
89,Kingsland,Row / Townhouse,House,"$250,000-$500,000","$500,000-$750,000",4 bedrooms,3 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,Coffee Shop,Sushi Restaurant,Gas Station,Burger Joint,Restaurant,25-50
93,Lincoln Park,Apartment,Row / Townhouse,"<=$250,000","$250,000-$500,000",1 bedrooms,3 bedrooms,1 bathrooms,3 bathrooms,500-1000 sqft,1000-2000 sqft,Coffee Shop,Pizza Place,Food Court,Sandwich Place,Bar,25-50
130,Ramsay,House,Duplex,"$750,000-$1,000,000","$250,000-$500,000",4 bedrooms,5 bedrooms,4 bathrooms,2 bathrooms,1000-2000 sqft,2000+ sqft,Restaurant,Coffee Shop,Brewery,Chinese Restaurant,Italian Restaurant,25-50


**Cluster 2**

Summary: Buyers interested in this cluster would be looking for a smaller less expensive house (although townhouses and apartments are options as well) under 500,000 with options under 250,000. Most homes are under 2000 sqft with 3-4 bedrooms and 2-3 bathrooms. Venue accessibility is lower with less than 25 nearby venues.

In [178]:
#cluster 2
final_merged.loc[final_merged['Cluster_Labels'] == 2, final_merged.columns[[0] + list(range(4, final_merged.shape[1]))]]

,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
0,Abbeydale,House,Mobile Home,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Wings Joint,Health & Beauty Service,Sandwich Place,Convenience Store,Construction & Landscaping,<25
1,Acadia,House,Apartment,"$250,000-$500,000","<=$250,000",4 bedrooms,3 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Sandwich Place,Pub,Fast Food Restaurant,Coffee Shop,Yoga Studio,<25
2,Albert Park/Radisson Heights,Apartment,Duplex,"<=$250,000","$250,000-$500,000",2 bedrooms,3 bedrooms,2 bathrooms,4 bathrooms,500-1000 sqft,1000-2000 sqft,Grocery Store,Indian Restaurant,Restaurant,Fried Chicken Joint,Fast Food Restaurant,<25
4,Applewood Park,House,Apartment,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Home Service,Coffee Shop,Park,Liquor Store,Food & Drink Shop,<25
5,Arbour Lake,House,Apartment,"$250,000-$500,000","<=$250,000",2 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Salon / Barbershop,Yoga Studio,Restaurant,Furniture / Home Store,Bookstore,<25
9,Bankview,Apartment,Row / Townhouse,"<=$250,000","$250,000-$500,000",2 bedrooms,1 bedrooms,1 bathrooms,4 bathrooms,1000-2000 sqft,500-1000 sqft,Pizza Place,Indian Restaurant,Coffee Shop,Convenience Store,Liquor Store,<25
11,Beddington Heights,Duplex,House,"$250,000-$500,000","<=$250,000",3 bedrooms,5 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,1000-2000 sqft,Bank,Coffee Shop,Grocery Store,Convenience Store,Liquor Store,<25
15,Bowness,House,Duplex,"$250,000-$500,000","$500,000-$750,000",3 bedrooms,4 bedrooms,2 bathrooms,4 bathrooms,1000-2000 sqft,500-1000 sqft,Pub,Stadium,Food & Drink Shop,Café,Coffee Shop,<25
16,Braeside,House,Row / Townhouse,"$250,000-$500,000","<=$250,000",4 bedrooms,3 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,Ice Cream Shop,Hockey Arena,Pub,Pizza Place,Convenience Store,<25
17,Brentwood,House,Apartment,"$250,000-$500,000","$500,000-$750,000",5 bedrooms,2 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,Coffee Shop,Fast Food Restaurant,Frozen Yogurt Shop,Bank,Grocery Store,<25


**Cluster 3**

Summary: Buyers interested in this cluster would be looking for apartments or townhouses under 500,000 and less than 1000 sqft. They commonly have 1-2 bedrooms and 1-2 bathrooms. These neighbourhoods have access to over 50 venues and looking at the map they are located in the city core.

In [176]:
#cluster 3
final_merged.loc[final_merged['Cluster_Labels'] == 3, final_merged.columns[[0] + list(range(4, final_merged.shape[1]))]]

,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
13,Beltline,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,1000-2000 sqft,Restaurant,Pub,Bar,Italian Restaurant,Coffee Shop,50+
35,Cliff Bungalow,Apartment,House,"<=$250,000","$750,000-$1,000,000",2 bedrooms,1 bedrooms,1 bathrooms,4 bathrooms,500-1000 sqft,1000-2000 sqft,Restaurant,Café,French Restaurant,Pizza Place,Gym / Fitness Center,50+
54,Downtown Commercial Core,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,1 bathrooms,2 bathrooms,500-1000 sqft,2000+ sqft,Coffee Shop,Restaurant,Pub,Bar,Sandwich Place,50+
55,Downtown East Village,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,<= 500 sqft,Coffee Shop,Vietnamese Restaurant,Theater,Italian Restaurant,Performing Arts Venue,50+
56,Downtown West End,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,1000-2000 sqft,Pub,Pizza Place,Sushi Restaurant,Bakery,Coffee Shop,50+
58,Eau Claire,Apartment,Row / Townhouse,"$250,000-$500,000","$500,000-$750,000",2 bedrooms,1 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Coffee Shop,Café,Sushi Restaurant,Restaurant,Hotel,50+
82,Hillhurst,Apartment,House,"$500,000-$750,000","$250,000-$500,000",2 bedrooms,4 bedrooms,4 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,American Restaurant,Pub,Restaurant,Sushi Restaurant,Gastropub,50+
94,Lower Mount Royal,Apartment,House,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,1 bathrooms,2 bathrooms,500-1000 sqft,1000-2000 sqft,Coffee Shop,Vietnamese Restaurant,Pub,Pizza Place,Grocery Store,50+
97,Manchester,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",2 bedrooms,1 bedrooms,2 bathrooms,1 bathrooms,500-1000 sqft,<= 500 sqft,Coffee Shop,Clothing Store,Electronics Store,Restaurant,Fast Food Restaurant,50+
109,Mission,Apartment,Row / Townhouse,"$250,000-$500,000","<=$250,000",1 bedrooms,2 bedrooms,1 bathrooms,2 bathrooms,500-1000 sqft,1000-2000 sqft,Italian Restaurant,Restaurant,Breakfast Spot,Yoga Studio,Pizza Place,50+


**Cluster 4**

Summary: Buyers interested in this cluster would be looking for houses or a townhouse priced under 750,000 with options under 500,000. These homes have 3-4 bedrooms and and 3-4 bathrooms. Homes in this cluster are commonly 1000-2000 sqft.  Venue accessibility is lower with under 25 nearby venues. Looking at the map many of these neighbourhoods lie near the edges of the city

In [177]:
#cluster 4
final_merged.loc[final_merged['Cluster_Labels'] == 4, final_merged.columns[[0] + list(range(4, final_merged.shape[1]))]]

,Neighbourhood,1st Most Common building_type,2nd Most Common building_type,1st Most Common price,2nd Most Common price,1st Most Common bedrooms,2nd Most Common bedrooms,1st Most Common bathrooms,2nd Most Common bathrooms,1st Most Common InteriorSize,2nd Most Common InteriorSize,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Most Common Venue Count
7,Auburn Bay,House,Apartment,"$250,000-$500,000","$500,000-$750,000",3 bedrooms,2 bedrooms,3 bathrooms,4 bathrooms,1000-2000 sqft,2000+ sqft,Construction & Landscaping,Other Repair Shop,Cosmetics Shop,Pizza Place,Food Truck,<25
14,Bonavista Downs,House,Row / Townhouse,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,<= 500 sqft,Business Service,Chinese Restaurant,Furniture / Home Store,Astrologer,Yoga Studio,<25
24,Castleridge,House,Duplex,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,3 bathrooms,1000-2000 sqft,500-1000 sqft,Convenience Store,Indian Restaurant,Bank,Grocery Store,Chinese Restaurant,<25
28,Chaparral,House,Row / Townhouse,"$250,000-$500,000","$500,000-$750,000",3 bedrooms,4 bedrooms,4 bathrooms,3 bathrooms,1000-2000 sqft,2000+ sqft,Coffee Shop,Home Service,Liquor Store,Gas Station,Sushi Restaurant,<25
31,Chinook Park,House,Apartment,"$500,000-$750,000","<=$250,000",5 bedrooms,4 bedrooms,3 bathrooms,1 bathrooms,1000-2000 sqft,500-1000 sqft,Pier,Accessories Store,Ice Cream Shop,Japanese Restaurant,Bakery,<25
33,Citadel,House,Apartment,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,3 bathrooms,4 bathrooms,1000-2000 sqft,500-1000 sqft,Gym,Coffee Shop,Market,Furniture / Home Store,Food Truck,<25
34,Cityscape,House,Row / Townhouse,"$250,000-$500,000","$500,000-$750,000",3 bedrooms,4 bedrooms,3 bathrooms,4 bathrooms,1000-2000 sqft,2000+ sqft,Home Service,Business Service,Ice Cream Shop,Yoga Studio,Food Truck,<25
36,Coach Hill,House,Row / Townhouse,"$250,000-$500,000","$500,000-$750,000",3 bedrooms,2 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,2000+ sqft,Gas Station,Vietnamese Restaurant,Convenience Store,Bar,Pizza Place,<25
38,Copperfield,House,Row / Townhouse,"$250,000-$500,000","<=$250,000",3 bedrooms,2 bedrooms,3 bathrooms,2 bathrooms,1000-2000 sqft,500-1000 sqft,Dance Studio,Trail,Convenience Store,Liquor Store,Food Court,<25
43,Coventry Hills,House,Row / Townhouse,"$250,000-$500,000","<=$250,000",3 bedrooms,4 bedrooms,2 bathrooms,4 bathrooms,1000-2000 sqft,500-1000 sqft,Coffee Shop,Furniture / Home Store,Shopping Mall,Construction & Landscaping,Lake,<25
